<a href="https://colab.research.google.com/github/tproffen/ORCSGirlsPython/blob/master/DoodleMining/QuickDrawDemos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Demonstration of using the Quickdraw dataset

* Quickdraw: https://quickdraw.withgoogle.com/ 
* Quickdraw dataset: https://quickdraw.withgoogle.com/data
* Quickdraw API: https://quickdraw.readthedocs.io/en/latest/index.html

* All valid categories are here : https://github.com/googlecreativelab/quickdraw-dataset/blob/master/categories.txt

**More reading**

* https://opensource.googleblog.com/2018/11/introducing-web-component-and-data-api-for-quick-draw.html

## Run these calls first

The two cells below need to be executed first. They install the Quickdraw API and load the needed Python modules.

In [ ]:
!pip3 install quickdraw

In [ ]:
from quickdraw import QuickDrawData, QuickDrawDataGroup, QuickDrawing
from IPython.display import Image, HTML
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from datetime import datetime
import random

## Simple retrieval of single doodles

Simple retrieval of a doogle and showing it as image.

Let's step it up and -redraw the image with custom background and color and add information about the drawing.

In [ ]:
what="spider"

qd = QuickDrawData()
spider=qd.get_drawing(what)

In [ ]:
spiderimg = spider.get_image(stroke_color=(255, 0, 0), stroke_width=3, bg_color=(255, 255, 255))

plt.figure()
plt.imshow(spiderimg)
plt.axis("off")
plt.show()

print ("Country: ",spider.countrycode)
print ("Date",datetime.fromtimestamp(spider.timestamp))
print ("Number of strokes: ",len(spider.strokes))

## Getting groups of doodles

Now we get a complete group of drawings. These are cached locally. Here we will display the first 10.

In [ ]:
# Loading doodles of pigs (default is 1000)

what="pig"
doodles = QuickDrawDataGroup(what)

In [ ]:
plt.figure(figsize=(20, 15))
# Now we can retrieve doodles from the group

for i in range(15):
  doodle = doodles.get_drawing()
  doodleimg = doodle.get_image(stroke_color=(0, 0, 255), stroke_width=3, bg_color=(255, 220, 220))
  plt.subplot(3, 5, i+1)
  plt.imshow(doodleimg)
  plt.title("Country: "+doodle.countrycode)
  plt.axis('off')

plt.show()


## Look for doodles by country

Since we know the country each doodle came from, we can compare drawing styles :)

In [ ]:
what = "bread"
countries = ["US", "JP", "FR", "CL"]
cols = 7  

all = QuickDrawDataGroup(what, max_drawings=5000, recognized=True)

In [ ]:
plt.figure(figsize=(20, 15))

index=1
for country in countries:

  doodles=all.search_drawings(countrycode=country)
  random.shuffle(doodles)
  print (country,len(doodles))

  for col in range(cols):
    img = doodles[col].get_image(stroke_color=(255,0,0), stroke_width=3, bg_color=(240,240,240))
    plt.subplot(len(countries), cols, index)
    plt.imshow(img)
    plt.title(country)
    plt.axis('off')
    index+=1;

plt.show()

## Let's try an animation of a doodle

In [ ]:
# Get a spider

what="spider"

qd = QuickDrawData()
spider=qd.get_drawing(what)

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.close()

ax.set_xlim((0, 255))
ax.set_ylim((0, 255))
ax.axis("off")

frames = spider.no_of_strokes

# animation function. This is called sequentially
def animate(i):
  xdata=[]
  ydata=[]
  for x, y in spider.strokes[i]:
    xdata.append(x)
    ydata.append(y)
  plot=ax.plot(xdata, ydata, lw=3)
  return plot

anim = animation.FuncAnimation(fig, animate, frames=frames, interval=500)

# Note: below is the part which makes it work on Colab
rc('animation', html='jshtml')
anim


## Let's draw many of the same doodles

Using transparency, we can see common features.

In [ ]:
# Get a group of spiders

what="chair"

doodles = QuickDrawDataGroup(what, recognized=True, max_drawings=50)

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
plt.close()

ax.set_xlim((0, 255))
ax.set_ylim((0, 255))
ax.axis("off")

frames = doodles.drawing_count

# animation function. This is called sequentially
def animate(i):
  doodle=doodles.get_drawing(i)
  xdata=[]
  ydata=[]
  for s in range(doodle.no_of_strokes):
    for x, y in doodle.strokes[s]:
      xdata.append(x)
      ydata.append(255-y)
  plot=ax.plot(xdata, ydata, 'black', lw=2, alpha=0.2)
  return plot

anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100)

# Note: below is the part which makes it work on Colab
rc('animation', html='jshtml')
anim